## Indexes:
- $ M = \{0, 1, 2\} $: Mills  

- $ W = \{0, \ldots, 3\} $: Warehouses 

- $ C = \{0, \ldots, 12\} $: Customer areas

## Parameters:
- $ d_j $: demand in customer area $ j $  
- $ p_i $: production capacity at mill $ i $  
- $ e_i $: extra production capacity at mill $ i $  
- $ c_{ik}^a $: transportation costs per ton from mill $ i $ to warehouse $ k $  
- $ c_{kj}^b $: transportation costs per ton from warehouse $ k $ to customer area $ j $
- $ c_i^e $: cost of using the extra production capacity at mill $ i $

## Decision Variables:
- $ x_{ik} $: Tons transported from mill $ i $ to warehouse $ k $  
- $ y_{kj} $: Tons transported from warehouse $ k $ to customer area $ j $
- $ z_i $: Use extra capacity at mill $ i $  

## Objective Function:
Minimize $ \sum_{i \in M} \left( \sum_{k \in W} c_{ik}^a x_{ik} + \sum_{j \in C} c_{kj}^b y_{kj} + z_i c_i^e \right) $

## Constraints:
- Production Capacity: $ \sum_{k \in W} x_{ik} \leq p_i + z_i e_i, \forall i \in M $  
- Balance at Warehouses: $ \sum_{i \in M} x_{ik} = \sum_{j \in C} y_{kj}, \forall k \in W $  
- Demand Satisfaction: $ \sum_{k \in W} y_{kj} = d_j, \forall j \in C $  
- Non-negativity: $ x_{ik} \geq 0, \forall i \in M, k \in W $; $ y_{kj} \geq 0, \forall k \in W, j \in C $
- Binary Decision: $ z_i \in \{0, 1\}, \forall i \in M $

Unfortunately, the Python implementation of the model was not completely finished before the analyst responsible for its implementation broke her leg in a weird skiing accident in the Alps. It is your job to finish the code and run it to identify the optimal transportation routes as well as the mills in which extra capacity should be utilized.


In [2]:
# Press shift+enter to run code in a cell

# Import PuLP library; if there is an error message type "pip install pulp" + shift-enter
from pulp import *

#Indices
Mills = list(range(3))
Warehouses =  list(range(4))
Customer_areas = list(range(13))

#Parameters
Prod_max=[9900, 2100, 4200] #Production capacities
Prod_extra = [1100,1400,1500] #Extra capacities
Demand = [650,260,650,130,780,2500,910,3120,910,3640,1040,1650,1780] #Customer area demands
Transp_cost_1 = [[53, 95, 136, 160],
            [60, 120, 132, 140],
            [210, 190, 89, 71]] #Transportation costs from mills to warehouses
Transp_cost_2 = [[100, 120, 150, 160, 300, 310, 340, 490, 430,360, 280, 350, 200],
            [200, 240, 280, 310, 280, 400, 440, 410, 380, 190, 80, 150, 90],
            [280, 260, 320, 400, 140, 319, 290, 240, 230, 80, 60, 60, 160],
            [320, 280, 200, 130, 130, 70, 100, 90, 100, 190, 370, 320, 390]] #Transportation costs from warehouses to customers
Extra_cost = [300000,400000,450000] #Extra capacity costs

model = LpProblem("PP_MILP_model", LpMinimize) #Create a optimization model (minimization)
X=[[LpVariable("x_"+str(i)+str(k),0,None) for k in Warehouses] for i in Mills] #Create decision variables x_ik
Y=[[LpVariable("y_"+str(k)+str(j),0,None) for j in Customer_areas] for k in Warehouses] #Create decision variables y_kj
Z=[LpVariable("z_"+str(i),cat="Binary") for i in Mills ] #Create decision variables z_i

#Set a temporary objective function for testing purposes
# model += (lpSum([1*X[i][k] for i in Mills for k in Warehouses]) + lpSum([1*Y[k][j] for k in Warehouses for j in Customer_areas]) + lpSum([1*Z[i] for i in Mills]), "Testing_objective_function")

#TODO: add objective function and constraints here after back from Alps!:

# YOUR CODE STARTS HERE


# Objective Function
model += (lpSum([Transp_cost_1[i][k] * X[i][k] for i in Mills for k in Warehouses]) +
          lpSum([Transp_cost_2[k][j] * Y[k][j] for k in Warehouses for j in Customer_areas]) +
          lpSum([Extra_cost[i] * Z[i] for i in Mills]), "Total_Cost")

# Constraints

# Production Capacity Constraint
for i in Mills:
    model += (lpSum([X[i][k] for k in Warehouses]) <= Prod_max[i] + (Prod_extra[i] * Z[i]), f"Prod Capacity {i}")

# Supply-Demand Balance at Warehouses
for k in Warehouses:
    model += (lpSum([X[i][k] for i in Mills]) == lpSum([Y[k][j] for j in Customer_areas]), f"Supply Demand Warehouse {k}")

# Demand Satisfaction
for j in Customer_areas:
    model += (lpSum([Y[k][j] for k in Warehouses]) == Demand[j], f"Demand Customer Area {j}")


# YOUR CODE ENDS HERE

model.solve() #Solve the optimal solution to model

#Print optimal objective function value:
print("Optimal total cost "+str(model.objective.value())+" .")
   
                     
#Print optimal decision variable values: 
for i in Mills:
    print("------")
    if (Z[i].varValue==1):
        print("Purchase extra capacity for mill "+str(i)) 
    for k in Warehouses:
        if (X[i][k].varValue>0):
            print("Transport "+str(X[i][k].varValue)+" tons from mill "+str(i)+" to warehouse "+str(k)+".") 
    

print("---------------")
for k in Warehouses:
    for j in Customer_areas: 
        if (Y[k][j].varValue>0):
            print("Transport "+str(Y[k][j].varValue)+" tons from warehouse "+str(k)+" to customer area "+str(j)+".") 
    print("---")
    

Optimal total cost 4174230.0 .
------
Purchase extra capacity for mill 0
Transport 1690.0 tons from mill 0 to warehouse 0.
Transport 2820.0 tons from mill 0 to warehouse 1.
Transport 5710.0 tons from mill 0 to warehouse 2.
------
Transport 2100.0 tons from mill 1 to warehouse 3.
------
Purchase extra capacity for mill 2
Transport 5700.0 tons from mill 2 to warehouse 3.
---------------
Transport 650.0 tons from warehouse 0 to customer area 0.
Transport 260.0 tons from warehouse 0 to customer area 1.
Transport 650.0 tons from warehouse 0 to customer area 2.
Transport 130.0 tons from warehouse 0 to customer area 3.
---
Transport 1040.0 tons from warehouse 1 to customer area 10.
Transport 1780.0 tons from warehouse 1 to customer area 12.
---
Transport 420.0 tons from warehouse 2 to customer area 4.
Transport 3640.0 tons from warehouse 2 to customer area 9.
Transport 1650.0 tons from warehouse 2 to customer area 11.
---
Transport 360.0 tons from warehouse 3 to customer area 4.
Transport 250